In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import numpy as np
import requests

In [4]:
x = requests.get('https://blocks.flashbots.net/v1/blocks') # just an example of loading in json via GET request

In [7]:
data = x.json()

In [3]:
with open('transaction_history_1000.json') as json_file: # how you would load in an already uploaded json file
    data = json.load(json_file)

In [32]:
blockdf2 = pd.DataFrame(data['blocks'])

In [33]:
blockdf2.head()

,block_number,miner_reward,miner,coinbase_transfers,gas_used,gas_price,transactions
0,15499457,3656032165854100,0x00192Fb10dF37c9FB26829eb2CC623cd1BF599E8,771218621941000,690527,5294553530,[{'transaction_hash': '0xdacbf1fc6c24346055bb3...
1,15499461,27818699083702895,0x2A20380DcA5bC24D052acfbf79ba23e988ad0050,8447958435705928,4305998,6460453322,[{'transaction_hash': '0x01ae9975f00765657ff12...
2,15499462,71185786003524985,0x829BD824B016326A401d083B33D092293333A830,3658525950107255,782759,90942149503,[{'transaction_hash': '0x5bb534814076581f36315...
3,15499465,30407951822936158,0xEA674fdDe714fd979de3EdF0F56AA9716B898ec8,25686584009236362,2024395,15020760189,[{'transaction_hash': '0xde5c5aaa70d1126b29671...
4,15499466,4807954373211815,0x829BD824B016326A401d083B33D092293333A830,2319549279398094,353089,13616834206,[{'transaction_hash': '0x45cb7711ccd0ab3e5145b...


In [36]:
gas_used = list(blockdf2.groupby('block_number')['gas_used'].sum())  # total gas used per block column

In [37]:
gas_price = list(blockdf2.groupby('block_number')['gas_price'].sum()) #total gas price per block column

In [55]:
final_df = pd.DataFrame(blockdf2.groupby('block_number')['miner_reward'].sum()) #quantifies raw MEV per block (in wei)

In [56]:
final_df['gas_used'] = gas_used        # adding columns into final df
final_df['gas_price'] = gas_price

In [57]:
final_df.head()  # propose using gas price as alternative to fluctuating eth price for time being, need to possibly query separate api with number of different oracles to obtain accurate eth price data per block

,miner_reward,gas_used,gas_price
block_number,,,
15499457,3656032165854100,690527,5294553530
15499461,27818699083702895,4305998,6460453322
15499462,71185786003524985,782759,90942149503
15499465,30407951822936158,2024395,15020760189
15499466,4807954373211815,353089,13616834206


In [58]:
def condition(x):
    if 0 <= x <= 4.32*(10**14):
         return 1
    elif 4.32*(10**14) < x <= 3.39*(10**15):
        return 2
    elif x > 3.39*(10**15):
        return 3

In [60]:
final_df['miner_reward'] = final_df['miner_reward'].astype('float') # change column data type so it will work with lambda condition

In [61]:
final_df.dtypes

miner_reward    float64
gas_used          int64
gas_price        object
dtype: object

In [62]:
final_df['MEV_category'] = final_df['miner_reward'].apply(condition) # new MEV category column

In [63]:
final_df.reset_index() #use this if you want to remove the remove the index - use 'inplace=True' as argument

,block_number,miner_reward,gas_used,gas_price,MEV_category
0,15499457,3.656032e+15,690527,5294553530,3
1,15499461,2.781870e+16,4305998,6460453322,3
2,15499462,7.118579e+16,782759,90942149503,3
3,15499465,3.040795e+16,2024395,15020760189,3
4,15499466,4.807954e+15,353089,13616834206,3
...,...,...,...,...,...
95,15499672,2.107610e+16,850663,24776091493,3
96,15499673,3.021379e+16,1327213,22764835285,3
97,15499674,5.518681e+16,1028814,53641194194,3
98,15499675,1.739566e+16,1358972,12800598568,3


In [43]:
blockdf.to_csv('final_df.csv') # how you would convert final dataframe to csv